In [1]:
import sqlalchemy as db
import pymysql
import pandas as pd
import pymysql.cursors

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import datetime 
import pandas as pd
import datetime

import os
import folium
import datetime 
from folium import plugins
from branca.colormap import linear
from shapely.geometry import Point, shape
from folium import plugins


import geopandas as gpd
import branca.colormap as cm
import json 

In [3]:
## importing packages
import lightgbm as lgb
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)
import datetime
%matplotlib inline
import re


In [4]:

# visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from plotnine import *
import plotly.express as px
import folium

from IPython.display import Javascript
from IPython.core.display import display, HTML

In [5]:
conn = pymysql.connect(host ='localhost',
                      user= 'root', 
                      password='root',
                      db= 'covid19',
                      port = 3306)
conn

In [116]:
donnees_covid_europe= pd.read_sql("SELECT * FROM covid19.donnees_covid_europe", con=conn)


In [117]:
donnees_covid_europe['Active'] = donnees_covid_europe['confirmed'] - donnees_covid_europe['Deaths'] - donnees_covid_europe['Recovered']

In [118]:
donnees_covid_europe[['confirmed', 'Deaths', 'Recovered', 'Active']] = donnees_covid_europe[['confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)
donnees_covid_europe['Recovered'] = donnees_covid_europe['Recovered'].astype(int)

In [99]:
# Grouped by day, country
# =======================

full_grouped = donnees_covid_europe.groupby(['date', 'Country'])['confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()


In [12]:
temp = donnees_covid_europe[donnees_covid_europe['date'] == max(donnees_covid_europe['date'])]

m = folium.Map(location=[34.553127, 20.048012], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=6)

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['latitude'], temp.iloc[i]['longitude']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['confirmed'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Deaths']),
        radius=int(temp.iloc[i]['confirmed'])**0.5).add_to(m)
m
m.save(os.path.join('results', 'planing1.html'))

In [44]:
formated_gdf = donnees_covid_europe.groupby(['date', 'Country'])['confirmed', 'Deaths'].max()
formated_gdf = formated_gdf.reset_index()
formated_gdf['date'] = pd.to_datetime(formated_gdf['date'])
formated_gdf['date'] = formated_gdf['date'].dt.strftime('%m/%d/%Y')
formated_gdf['size'] = formated_gdf['confirmed'].pow(0.3)

fig = px.scatter_geo(formated_gdf, locations="Country", locationmode='country names', 
                     color="confirmed", size='size', hover_name="Country", 
                     range_color= [0, max(formated_gdf['confirmed'])+2], animation_frame="date", 
                     title='Spread over time', projection="natural earth")
fig.update(layout_coloraxis_showscale=False)
# fig.show()
fig.write_html("results/file.html")


In [96]:
c_lat = donnees_covid_europe[donnees_covid_europe['date'] == max(donnees_covid_europe['date'])].reset_index()
pays_dc  = c_lat.groupby(['Country'])['Deaths'].sum().reset_index()


In [102]:
#lire fichier geoson pays euro
data_geo = gpd.read_file('european-union-countries.geojson', driver='GeoJSON')
data_geo = data_geo[["su_a3","name", "geometry"]]
data_geo = data_geo.rename(columns={'su_a3': 'code','name': 'Country'})


In [90]:
df_heatmap = pd.merge(data_geo, pays_dc, on='Country')

In [94]:


map_osm = folium.Map(location=[50, 10] , zoom_start=3.5)
couleur = linear.YlGnBu_09.scale(min(df_heatmap['Deaths']), max(df_heatmap['Deaths']))
style_function = lambda x: {'fillColor': couleur(x['properties']['Deaths']),'color':'black','weight': 1.5,'fillOpacity': 0.7}
folium.GeoJson(df_heatmap,style_function=style_function,tooltip=folium.GeoJsonTooltip(
        fields=['Country','code','Deaths'],
        aliases=['Country','code', 'Deaths',],
        localize=True
    )
).add_to(map_osm)
couleur.add_to(map_osm)
couleur.add_to(map_osm)
map_osm.save(os.path.join('results', 'Plugins_Décés_par_country.html'))


In [105]:
# new cases ======================================================
temp = full_grouped.groupby(['Country', 'date', ])['confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()

mask = temp['Country'] != temp['Country'].shift(1)
temp.loc[mask, 'confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
# # renaming columns
temp.columns = ['Country', 'date', 'New cases', 'New deaths', 'New recovered']

# =================================================================
temp


,Country,date,New cases,New deaths,New recovered
0,Albania,2020-01-22,NaN,NaN,NaN
1,Albania,2020-01-23,0.0,0.0,0.0
2,Albania,2020-01-24,0.0,0.0,0.0
3,Albania,2020-01-25,0.0,0.0,0.0
4,Albania,2020-01-26,0.0,0.0,0.0
...,...,...,...,...,...
3995,United Kingdom,2020-05-21,2627.0,338.0,18.0
3996,United Kingdom,2020-05-22,3298.0,351.0,8.0
3997,United Kingdom,2020-05-23,2960.0,282.0,7.0
3998,United Kingdom,2020-05-24,2412.0,118.0,2.0


In [106]:
# merging new values
full_grouped_new = pd.merge(full_grouped, temp, on=['Country', 'date'])
# # # filling na with 0
full_grouped_new = full_grouped_new.fillna(0)

# # # fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped_new[cols] = full_grouped_new[cols].astype('int')

full_grouped_new['New cases'] = full_grouped_new['New cases'].apply(lambda x: 0 if x<0 else x)
full_grouped_new

,date,Country,confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Albania,0,0,0,0,0,0,0
1,2020-01-22,Belgium,0,0,0,0,0,0,0
2,2020-01-22,Bosnia and Herzegovina,0,0,0,0,0,0,0
3,2020-01-22,Bulgaria,0,0,0,0,0,0,0
4,2020-01-22,Croatia,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
3995,2020-05-25,Sweden,33843,4029,4971,24843,384,31,0
3996,2020-05-25,Switzerland,30746,1913,28200,633,10,7,100
3997,2020-05-25,Turkey,157814,4369,120015,33430,987,29,1321
3998,2020-05-25,Ukraine,21245,623,7234,13388,259,6,126


In [113]:
# Day wise
# ========

# table
day_wise = full_grouped_new.groupby('date')['confirmed', 'Deaths', 'Recovered', 'Active', 'New cases', 'New deaths'].sum().reset_index()

# number cases per 100 cases
day_wise['Deaths / 100 Cases'] = round((day_wise['Deaths']/day_wise['confirmed'])*100, 2)
day_wise['Recovered / 100 Cases'] = round((day_wise['Recovered']/day_wise['confirmed'])*100, 2)
day_wise['Deaths / 100 Recovered'] = round((day_wise['Deaths']/day_wise['Recovered'])*100, 2)

# no. of countries
# day_wise['No. of countries'] = full_grouped_new[full_grouped_new['confirmed']!=0].groupby('date')['Country'].unique().apply(len).values

# # fillna by 0
cols = ['Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered']
day_wise[cols] = day_wise[cols].fillna(0)

day_wise

,date,confirmed,Deaths,Recovered,Active,New cases,New deaths,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered
0,2020-01-22,0,0,0,0,0,0,0.00,0.00,0.00
1,2020-01-23,0,0,0,0,0,0,0.00,0.00,0.00
2,2020-01-24,2,0,0,2,2,0,0.00,0.00,0.00
3,2020-01-25,3,0,0,3,1,0,0.00,0.00,0.00
4,2020-01-26,3,0,0,3,0,0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
120,2020-05-21,1560598,166197,757572,636829,8558,916,10.65,48.54,21.94
121,2020-05-22,1571024,167582,772481,630961,10426,1385,10.67,49.17,21.69
122,2020-05-23,1579115,168294,778398,632423,8091,712,10.66,49.29,21.62
123,2020-05-24,1586928,168868,793565,624495,7813,574,10.64,50.01,21.28


In [122]:

act = '#4a47a3'
rec = '#12cad6'
dth = '#f0134d'

tem = donnees_covid_europe.groupby('date')['confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
tem = tem[tem['date']==max(tem['date'])].reset_index(drop=True)

tm = tem.melt(id_vars="date", value_vars=['Active', 'Deaths', 'Recovered'])
fig = px.treemap(tm, path=["variable"], values="value", height=225, width=1200,
                 color_discrete_sequence=[act, rec, dth])
fig.data[0].textinfo = 'label+text+value'
fig.show()


In [123]:
temp = donnees_covid_europe.groupby('date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.area(temp, x="date", y="Count", color='Case', height=600,
             title='Cases over time', color_discrete_sequence = [rec, dth, act])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [128]:
# Country wise

# ============

# getting latest values
country_wise = full_grouped_new[full_grouped_new['date']==max(full_grouped_new['date'])].reset_index(drop=True).drop('date', axis=1)
country_wise.head(3)

,Country,confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,Albania,1004,32,795,177,6,0,6
1,Belgium,57342,9312,15297,32733,250,32,25
2,Bosnia and Herzegovina,2406,146,1696,564,5,2,16


In [133]:

# group by country
country_wise = country_wise.groupby('Country')['confirmed', 'Deaths', 'Recovered', 'Active', 'New cases'].sum().reset_index()

# per 100 cases
country_wise['Deaths / 100 Cases'] = round((country_wise['Deaths']/country_wise['confirmed'])*100, 2)
country_wise['Recovered / 100 Cases'] = round((country_wise['Recovered']/country_wise['confirmed'])*100, 2)
country_wise['Deaths / 100 Recovered'] = round((country_wise['Deaths']/country_wise['Recovered'])*100, 2)

cols = ['Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered']
country_wise[cols] = country_wise[cols].fillna(0)

country_wise


,Country,confirmed,Deaths,Recovered,Active,New cases,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered
0,Albania,1004,32,795,177,6,3.19,79.18,4.03
1,Belgium,57342,9312,15297,32733,250,16.24,26.68,60.87
2,Bosnia and Herzegovina,2406,146,1696,564,5,6.07,70.49,8.61
3,Bulgaria,2433,130,862,1441,6,5.34,35.43,15.08
4,Croatia,2244,100,2035,109,0,4.46,90.69,4.91
5,Denmark,11586,563,10162,861,27,4.86,87.71,5.54
6,Estonia,1824,65,1538,221,1,3.56,84.32,4.23
7,Finland,6599,308,5100,1191,20,4.67,77.28,6.04
8,France,183067,28460,65317,89290,358,15.55,35.68,43.57
9,Germany,180600,8309,161199,11092,272,4.60,89.26,5.15


In [161]:
popp = pd.read_csv('population_by_country_2020.csv')
# select only population
pop = popp.iloc[:, :2]
pop.columns = ['Country', 'Population']
pays_europe =['Italy' , 'Portugal' , 'Spain' , 'France' , 'Netherlands' , 'Germany' , 'Poland' , 'Ireland', 'Ukraine', 'Romania' , 'Switzerland' , 'Albania' ,  'Austria,Belarus', 'Belgium' ,'Bosnia and Herzegovina', 'Bulgaria','Croatia','Denmark' ,'Estonia', 'Finland','Greece','Hungary', 'Iceland', 'Latvia' ,'Lithuania','Malta' ,'Moldova' , 'Montenegro','Norway', 'North Macedonia', 'Russia', 'Serbia', 'Slovakia' ,'Slovenia' , 'Sweden','Turkey'  ,'Ukraine' , 'United Kingdom']
pop = pop[pop.Country.isin(pays_europe)]

#merge data pop avec country_wise

df_country_wise = pd.merge(country_wise, pop, on='Country', how='left')

# Cases per population
df_country_wise['Cases / Million People'] = round((df_country_wise['confirmed'] / df_country_wise['Population']) * 1000000)


In [172]:
from datetime import timedelta
#
today = full_grouped[full_grouped['date']==max(full_grouped['date'])].reset_index(drop=True).drop('date', axis=1)[['Country', 'confirmed']]
last_week = full_grouped[full_grouped['date']==max(full_grouped['date'])-timedelta(days=7)].reset_index(drop=True).drop('date', axis=1)[['Country', 'confirmed']]
tempp = pd.merge(today, last_week, on='Country', suffixes=(' today', ' last week'))
tempp.head()

,Country,confirmed today,confirmed last week
0,Albania,1004,948
1,Belgium,57342,55559
2,Bosnia and Herzegovina,2406,2304
3,Bulgaria,2433,2235
4,Croatia,2244,2228


In [183]:
# confirmed - deaths
fig_c = px.bar(country__wise.sort_values('confirmed').tail(15), x="confirmed", y="Country", 
               text='confirmed', orientation='h', color_discrete_sequence = [act])
fig_d = px.bar(country__wise.sort_values('Deaths').tail(15), x="Deaths", y="Country", 
               text='Deaths', orientation='h', color_discrete_sequence = [dth])

# # recovered - active
fig_r = px.bar(country__wise.sort_values('Recovered').tail(15), x="Recovered", y="Country", 
               text='Recovered', orientation='h', color_discrete_sequence = [rec])
fig_a = px.bar(country__wise.sort_values('Active').tail(15), x="Active", y="Country", 
               text='Active', orientation='h', color_discrete_sequence = ['#333333'])

In [184]:
# death - recoverd / 100 cases
fig_dc = px.bar(country__wise.sort_values('Deaths / 100 Cases').tail(15), x="Deaths / 100 Cases", y="Country", 
               text='Deaths / 100 Cases', orientation='h', color_discrete_sequence = ['#f38181'])
fig_rc = px.bar(country__wise.sort_values('Recovered / 100 Cases').tail(15), x="Recovered / 100 Cases", y="Country", 
               text='Recovered / 100 Cases', orientation='h', color_discrete_sequence = ['#a3de83'])



In [185]:
# new cases - cases per million people
fig_nc = px.bar(country__wise.sort_values('New cases').tail(15), x="New cases", y="Country", 
               text='New cases', orientation='h', color_discrete_sequence = ['#c61951'])
temp = country__wise[country__wise['Population']>1000000]
fig_p = px.bar(temp.sort_values('Cases / Million People').tail(15), x="Cases / Million People", y="Country", 
               text='Cases / Million People', orientation='h', color_discrete_sequence = ['#741938'])


In [182]:
# tempp = tempp[['Country/Region', 'Confirmed last week']]
tempp['1 week change'] = tempp['confirmed today'] - tempp['confirmed last week']

temp = tempp[['Country', 'confirmed last week', '1 week change']]

country__wise = pd.merge(df_country_wise, tempp, on='Country')

country__wise['1 week % increase'] = round(country__wise['1 week change']/country__wise['confirmed last week']*100, 2)

country__wise.head()

,Country,confirmed,Deaths,Recovered,Active,New cases,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Population,Cases / Million People,confirmed today,confirmed last week,1 week change,1 week % increase
0,Albania,1004,32,795,177,6,3.19,79.18,4.03,2878420,349.0,1004,948,56,5.91
1,Belgium,57342,9312,15297,32733,250,16.24,26.68,60.87,11579477,4952.0,57342,55559,1783,3.21
2,Bosnia and Herzegovina,2406,146,1696,564,5,6.07,70.49,8.61,3284806,732.0,2406,2304,102,4.43
3,Bulgaria,2433,130,862,1441,6,5.34,35.43,15.08,6958627,350.0,2433,2235,198,8.86
4,Croatia,2244,100,2035,109,0,4.46,90.69,4.91,4110214,546.0,2244,2228,16,0.72


In [189]:
# week change, percent increase
fig_wc = px.bar(country__wise.sort_values('1 week change').tail(15), x="1 week change", y="Country", 
               text='1 week change', orientation='h', color_discrete_sequence = ['#004a7c'])
temp = country__wise[country__wise['confirmed']>100]
fig_pi = px.bar(temp.sort_values('1 week % increase').tail(15), x="1 week % increase", y="Country", 
               text='1 week % increase', orientation='h', color_discrete_sequence = ['#005691'], 
                hover_data=['confirmed last week', 'confirmed'])


In [191]:
# plot
from plotly.subplots import make_subplots

fig = make_subplots(rows=5, cols=2, shared_xaxes=False, horizontal_spacing=0.14, vertical_spacing=0.08,
                    subplot_titles=('Confirmed cases', 'Deaths reported', 'Recovered', 'Active cases', 
                                    'Deaths / 100 cases', 'Recovered / 100 cases', 'New cases', 
                                    'Cases / Million People', '1 week increase', '1 week % increase'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)
fig.add_trace(fig_r['data'][0], row=2, col=1)
fig.add_trace(fig_a['data'][0], row=2, col=2)

fig.add_trace(fig_dc['data'][0], row=3, col=1)
fig.add_trace(fig_rc['data'][0], row=3, col=2)
fig.add_trace(fig_nc['data'][0], row=4, col=1)
fig.add_trace(fig_p['data'][0], row=4, col=2)
fig.add_trace(fig_wc['data'][0], row=5, col=1)
fig.add_trace(fig_pi['data'][0], row=5, col=2)


fig.update_layout(height=3000)
fig.write_html("results/file_analyse.html")

In [193]:
fig = px.bar(full_grouped, x="date", y="Deaths", color='Country', height=600,
             title='Deaths', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.show()
fig.write_html("results/file_Deaths.html")